#Instalação

In [ ]:
# reading daaset from Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install emoji
!pip install simplejson
!pip install wmd
!pip install kneed

#Metrics
!pip install octis

!pip install tmplot
!pip install git+https://github.com/maximtrp/tmplot.git

In [ ]:
 # Packages to store and manipulate data
import pandas as pd
import numpy as np
# Example function using numpy:
from numpy import dot
from numpy.linalg import norm
import math
import os
import json
import simplejson as json
import csv
import string
import glob
import random
import time
import math
from datetime import datetime
pd.set_option('max_colwidth', 200)
 
# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import pickle
import math
import wmd
from scipy import spatial
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
from collections import Counter
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
 
# Packages
import emoji
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
import operator
from operator import itemgetter
from tqdm import tqdm
from collections import defaultdict
from textblob import TextBlob
import sys
import itertools
import spacy
from scipy.sparse import csr_matrix, issparse  # , todense
import sys
from collections import defaultdict
import multiprocessing
from pprint import pprint
 
# Gensim
import gensim
import gensim.downloader
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim import models
from gensim import corpora
from gensim.models import CoherenceModel
import gensim.downloader as api
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.models.ldamulticore import LdaMulticore
 
# Model building package: Sklearn
import sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
# Cluster documents: Sklearn
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

#Metrics
from octis.evaluation_metrics.metrics import AbstractMetric
from octis.dataset.dataset import Dataset
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import octis.configuration.citations as citations
import numpy as np
import itertools
from scipy import spatial
from sklearn.metrics import pairwise_distances
from operator import add
from octis.evaluation_metrics.rbo import rbo
import gensim.downloader as api
from gensim.models import KeyedVectors
from octis.evaluation_metrics.diversity_metrics import WordEmbeddingsInvertedRBO, \
    WordEmbeddingsInvertedRBOCentroid, InvertedRBO
from itertools import combinations
from scipy.spatial.distance import cosine

#Importar Dados

In [ ]:
df = pd.read_json('dados_fevereiro_semana_1.json')
#Tokenizar

new_tweet = []
for tweet in df.clean_tweet:
    tweet_len = [word for word in tweet.split(' ') if len(word) >= 3]
    tweet_string = ' '.join(tweet_len)
    new_tweet.append(tweet_string)
df['clean_tweet'] = new_tweet

stop = stopwords.words('english')
df['clean_tweet_tokenize'] = df['clean_tweet'].apply(lambda x: [item for item in str(x).split(' ')])
df.dropna()
df

#Modelagem de Tópico

In [ ]:
# GENSIM
class LdaModel:
    
    def __init__(self, column, num_topics):
        self.column = column
        self.num_topics = num_topics
        self.__preparing_application()
        self.__applying_topic_modelling_lda()
        self.__save_the_model()
        
    def __preparing_application(self):
        # Create Dictionary
        self.id2word = corpora.Dictionary(self.column.to_numpy())
        # Create Corpus: Term Document Frequency
        self.corpus = [self.id2word.doc2bow(text) for text in self.column]
        
    def __applying_topic_modelling_lda(self):
        # Build LDA model
        self.lda_model = gensim.models.ldamodel.LdaModel(corpus=self.corpus,
                                                   id2word=self.id2word,
                                                   num_topics=self.num_topics, 
                                                   random_state=42) 
        
    def get_lda(self, topic_number):
        # Shows the weight of each word in the topic
        return self.lda_model.print_topics(topic_number)

    def get_topics_lda_set_tokes(self):
        self.dic_topic_set = {}
        for index, topic in self.lda_model.show_topics(formatted=False, num_words= 30): #More tokens
            name = 'Topic ' + str(index)
            self.dic_topic_set[name] = ([w[0] for w in topic])
        return self.dic_topic_set

    def get_display_topics_lda_set_tokes(self):
        self.get_topics_lda_set_tokes()
        dataframe_topics = pd.DataFrame(data=self.dic_topic_set)
        return dataframe_topics

    def get_topics_lda(self):
        # Filtering for words 
        words = [re.findall(r'"([^"]*)"',t[1]) for t in self.lda_model.print_topics()]
        
        # Create Topics
        topics = [t[0:10] for t in (words)]
        self.dic_topic = {}
        self.list_topic = []
        # Getting the topics
        for id, t in enumerate(topics): 
          topic = 'Topic ' + str(id)
          self.list_topic.append(t)
          self.dic_topic[topic] = t
        
        return self.dic_topic

    def get_topic_in_list(self):
        self.get_topics_lda()
        return self.list_topic
      
    def get_display_topics_lda(self):
        self.get_topics_lda()
        dataframe_topics = pd.DataFrame(data=self.dic_topic)
        return dataframe_topics

    def __save_the_model(self):
        self.lda_model.save('lda.model_train')

    def plot(self):
        pyLDAvis.enable_notebook()
        lda_visualization = pyLDAvis.gensim_models.prepare(self.lda_model, self.corpus_select, self.id2word, sort_topics=False)
        return pyLDAvis.display(lda_visualization)

In [ ]:
column = df['clean_tweet_tokenize'] 
lda_model = LdaModel(column, 10)

#Tópicos

In [ ]:
lda_topic= lda_model.get_display_topics_lda()
lda_topic

In [ ]:
topics = lda_model.get_topic_in_list()

# Visualização

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
lda_model.plot()

# Métricas

In [ ]:
# cast tweets to numpy array
docs = df.clean_tweet_tokenize.to_numpy()
# create dictionary of all words in all documents
dictionary =  corpora.Dictionary(docs)
# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
cm_lda = CoherenceModel(topics=topics, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs, 
                          coherence='c_v')

print(cm_lda.get_coherence()) 

In [ ]:
npmi = CoherenceModel(topics=topics, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs,
                          coherence='c_npmi')

print(npmi.get_coherence())

In [ ]:
def get_word2index(list1, list2):
    words = set(list1)
    words = words.union(set(list2))
    word2index = {w: i for i, w in enumerate(words)}
    return word2index

In [ ]:
#Diversity Metrics:
class TopicDiversity(AbstractMetric):
    def __init__(self, topk=10):
        """
        Initialize metric
        Parameters
        ----------
        topk: top k words on which the topic diversity will be computed
        """
        AbstractMetric.__init__(self)
        self.topk = topk

    def score(self, model_output):
        """
        Retrieves the score of the metric
        Parameters
        ----------
        model_output : dictionary, output of the model
                       key 'topics' required.
        Returns
        -------
        td : score
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than ' + str(self.topk))
        else:
            unique_words = set()
            for topic in topics:
                unique_words = unique_words.union(set(topic[:self.topk]))
            td = len(unique_words) / (self.topk * len(topics))
            return td

In [ ]:
diversity = TopicDiversity()
diversity.score(topics)

In [ ]:
class InvertedRBO(AbstractMetric):
    def __init__(self, topk=10, weight=0.9):
        """
        Initialize metric Inverted Ranked-Biased Overlap
        :param topk: top k words on which the topic diversity will be computed
        :param weight: weight of each agreement at depth d. When set to 1.0, there is no weight, the rbo returns to
        average overlap. (default 0.9)
        """
        super().__init__()
        self.topk = topk
        self.weight = weight

    def score(self, model_output):
        """
        Retrieves the score of the metric
        :param model_output : dictionary, output of the model. the 'topics' key is required.
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')
        else:
            collect = []
            for list1, list2 in itertools.combinations(topics, 2):
                word2index = get_word2index(list1, list2)
                indexed_list1 = [word2index[word] for word in list1]
                indexed_list2 = [word2index[word] for word in list2]
                rbo_val = rbo(indexed_list1[:self.topk], indexed_list2[:self.topk], p=self.weight)[2]
                collect.append(rbo_val)
            return 1 - np.mean(collect)

In [ ]:
b = InvertedRBO()
b.score(topics)